In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_probability as tfp
import optuna
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
df = pd.read_csv(r"C:\Users\Robyi\Documents\Data Science Dataset\elecproduction.csv")
df.dropna(inplace=True)
df.drop_duplicates(inplace=True)
df.head()

In [ ]:
df.plot(figsize=(10, 4), title="Electricity Production Over Time")
plt.show()

In [ ]:
def create_features(df, lags=12):
    data = df.copy()
    for lag in range(1, lags + 1):
        data[f"lag_{lag}"] = df["IPG2211A2N"].shift(lag)
    return data.dropna()

df = create_features(df, lags=12)

In [ ]:
train_size = int(len(df) * 0.8)
train, test = df.iloc[:train_size], df.iloc[train_size:]

In [ ]:
scaler = StandardScaler()
train_scaled = scaler.fit_transform(train)
test_scaled = scaler.transform(test)

In [ ]:
X_train, y_train = train_scaled[:, 1:], train_scaled[:, 0]
X_test, y_test = test_scaled[:, 1:], test_scaled[:, 0]

In [ ]:
tfd = tfp.distributions

def create_bnn(hidden_units=16, learning_rate=0.01):
    model = tf.keras.Sequential([
        tfp.layers.DenseVariational(
            units=hidden_units,
            make_prior_fn=lambda: tfd.Normal(loc=0., scale=1.),
            make_posterior_fn=lambda: tfd.Normal(
                loc=tf.Variable(tf.random.normal([hidden_units])),
                scale=tf.nn.softplus(tf.Variable(tf.random.normal([hidden_units])))
            ),
            activation="relu"
        ),
        tfp.layers.DenseVariational(
            units=hidden_units,
            make_prior_fn=lambda: tfd.Normal(loc=0., scale=1.),
            make_posterior_fn=lambda: tfd.Normal(
                loc=tf.Variable(tf.random.normal([hidden_units])),
                scale=tf.nn.softplus(tf.Variable(tf.random.normal([hidden_units])))
            ),
            activation="relu"
        ),
        tfp.layers.DenseVariational(
            units=1,
            make_prior_fn=lambda: tfd.Normal(loc=0., scale=1.),
            make_posterior_fn=lambda: tfd.Normal(
                loc=tf.Variable(tf.random.normal([1])),
                scale=tf.nn.softplus(tf.Variable(tf.random.normal([1])))
            )
        )
    ])
    
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),
                  loss="mse",
                  metrics=["mae"])
    
    return model

In [ ]:
def objective(trial):
    hidden_units = trial.suggest_int("hidden_units", 8, 64)
    learning_rate = trial.suggest_float("learning_rate", 1e-4, 1e-2, log=True)

    model = create_bnn(hidden_units, learning_rate)
    
    history = model.fit(X_train, y_train, epochs=50, verbose=0, batch_size=16, validation_split=0.2)
    
    val_loss = history.history["val_loss"][-1]
    return val_loss

In [ ]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=10)

In [ ]:
best_params = study.best_params
print("Best Hyperparameters:", best_params)

In [ ]:
best_model = create_bnn(hidden_units=best_params["hidden_units"], learning_rate=best_params["learning_rate"])

history = best_model.fit(X_train, y_train, epochs=100, batch_size=16, verbose=1, validation_split=0.2)

In [ ]:
y_pred_samples = np.array([best_model(X_test) for _ in range(100)])

y_pred_mean = y_pred_samples.mean(axis=0).flatten()
y_pred_std = y_pred_samples.std(axis=0).flatten()

In [ ]:
mse = mean_squared_error(y_test, y_pred_mean)
r2 = r2_score(y_test, y_pred_mean)

print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"R-squared (R²): {r2:.4f}")

In [ ]:
y_test_actual = scaler.inverse_transform(np.column_stack([y_test, X_test]))[:, 0]
y_pred_actual = scaler.inverse_transform(np.column_stack([y_pred_mean, X_test]))[:, 0]
y_pred_std_actual = scaler.inverse_transform(np.column_stack([y_pred_std, X_test]))[:, 0]

plt.figure(figsize=(10, 5))
plt.plot(df.index[train_size:], y_test_actual, label="Actual", color="red", alpha=0.6)
plt.plot(df.index[train_size:], y_pred_actual, label="Predicted Mean", color="blue")
plt.fill_between(df.index[train_size:], 
                 y_pred_actual - 2 * y_pred_std_actual, 
                 y_pred_actual + 2 * y_pred_std_actual, 
                 color="blue", alpha=0.3, label="Uncertainty (95%)")
plt.xlabel("Time")
plt.ylabel("Electricity Production")
plt.legend()
plt.title("Bayesian Neural Network Time Series Forecasting with Uncertainty")
plt.show()

In [ ]:
last_data = df.iloc[-12:].values 

last_data_scaled = scaler.transform(last_data.reshape(1, -1))[:, 1:]

future_pred_samples = np.array([best_model(last_data_scaled) for _ in range(100)])

future_pred_mean = future_pred_samples.mean()
future_pred_std = future_pred_samples.std()

future_pred_actual = scaler.inverse_transform([[future_pred_mean] + list(last_data_scaled[0])])[0, 0]
future_pred_std_actual = scaler.inverse_transform([[future_pred_std] + list(last_data_scaled[0])])[0, 0]

print(f"🔹 Prediksi Produksi Listrik Periode Berikutnya: {future_pred_actual:.2f}")
print(f"🔹 Estimasi Ketidakpastian (±2σ): ±{2 * future_pred_std_actual:.2f}")